In [ ]:
%reload_ext autoreload
%autoreload 2

# All the packages are defined in the Config file
from model_packages import *

# All the variables are defined in the Config file
from model_config import *

# All the functions are defined in the Config file
from model_utils import *

# https://geographicdata.science/book/notebooks/10_clustering_and_regionalization.html

1.Read the locations of the sites

In [ ]:
# Ensure we read those sites which have gone through the pre-processing stage

data_processed_sites=pd.read_pickle('./data/static_and_dynamic_features.pkl')

data_processed_sites=data_processed_sites[~(data_processed_sites['counter'].isin(['Galley_Hall','Reigate_Fort']))].\
reset_index(drop=True)



2.Counters for each provider

In [ ]:
data_processed=data_processed_sites.copy()
data_processed=gpd.GeoDataFrame(data_processed[['site','provider','geometry']].drop_duplicates())

data_processed.geometry=data_processed.geometry.to_crs(crs_mtr).centroid

print(data_processed['provider'].value_counts())

data_processed.explore(column='provider',cmap='viridis',legend=True)

In [ ]:
# Visual of the visitation count time series for each provider

new_df=data_processed_sites[['people_counter_data','total_trip_count','site',\
                                                          'counter','provider','geometry','Date']]

new_df = new_df.groupby(['Date','provider'])['people_counter_data'].sum().unstack()

new_df.plot(style='-o')
plt.ylabel('People count')

3.Counters for each region

In [ ]:
# Counter locations overlap with regions

counters_locn=data_processed.copy()

counters_locn.rename(columns={'site':'counter'},inplace=True)


#counters_locn['provider'].value_counts()

regions=gpd.read_file('./data/regions/')

sites_per_region=gpd.overlay(counters_locn.to_crs(regions.crs),regions, how='intersection')

sites_per_region_count=sites_per_region.groupby(['RGN20NM','provider'])['counter'].count().reset_index()#.plot(kind='bar',stacked=True)


sites_per_region_count=sites_per_region_count.sort_values(by='counter').reset_index()

fig = px.bar(sites_per_region_count, x='RGN20NM', y="counter", color="provider",barmode='group')#,markers=True,symbol="season")

fig.update_traces(marker=dict(size=15,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(yaxis_title="Number of counters")

fig.show()

In [ ]:
sites_per_region.explore(column='RGN20NM')


In [ ]:
sites_per_region_count.groupby(['RGN20NM'])['counter'].sum().sort_values().plot(kind='barh')

4.Counters within protected landscapes

In [ ]:
#buffer around protected landscapes

counters_locn.geometry=counters_locn.geometry.buffer(5000)

In [ ]:
#protected landscapes: National Parks/AONB

np=gpd.read_file(national_park_data).to_crs(crs_mtr)

np['Type']='National park'

aonb=gpd.read_file('./data/AONB').to_crs(crs_mtr)

aonb['Type']='AONB'

area_of_intetest=gpd.GeoDataFrame(pd.concat([np[['name','geometry','Type']],\
                                             aonb[['name','geometry','Type']]], ignore_index=True))

ax=area_of_intetest.to_crs(crs_deg).plot(column='Type',legend=True)
cx.add_basemap(ax, crs = crs_deg, source = cx.providers.OpenStreetMap.Mapnik)

print(area_of_intetest['Type'].value_counts())

In [ ]:
# spatial overlap between counter locations and protected landscapes

sites_area_of_intetest=gpd.overlay(counters_locn.to_crs(area_of_intetest.crs),area_of_intetest, how='intersection')


sites_area_of_intetest.geometry=sites_area_of_intetest.geometry.centroid

In [ ]:
print(sites_area_of_intetest.groupby(['Type'])['counter'].count())

In [ ]:
print(sites_area_of_intetest.groupby(['Type','name'])['counter'].count())

In [ ]:
sites_protected_count=sites_area_of_intetest.groupby(['provider','Type'])['counter'].count().reset_index()

sites_protected_count


fig = px.bar(sites_protected_count, x='provider', y="counter", color="Type",barmode='group')#,markers=True,symbol="season")

fig.update_traces(marker=dict(size=15,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(yaxis_title="Number of counters")

fig.show()

In [ ]:
ax=area_of_intetest.to_crs(crs_deg).plot()
sites_area_of_intetest.to_crs(crs_deg).plot(ax=ax,column='Type',legend=True,cmap='rainbow')
cx.add_basemap(ax, crs = crs_deg, source = cx.providers.OpenStreetMap.Mapnik)

In [ ]:
sites_area_of_intetest.explore(column='name')


In [ ]:


sites_area_intrst=sites_area_of_intetest.groupby(['provider','name'])['counter'].count().reset_index()

fig = px.bar(sites_area_intrst, x='provider', y="counter", color="name",barmode='group')#,markers=True,symbol="season")

fig.update_traces(marker=dict(size=15,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.update_layout(yaxis_title="Number of counters")

fig.show()

5. Reading Census 2021 data

In [ ]:
oa_2021=gpd.read_file(data_folder_census+'oa_2021')

oa_2021=oa_2021[oa_2021['OA21CD'].str.lower().str.startswith('e')].reset_index(drop=True)


print(oa_2021.shape)

In [ ]:
df_densty=pd.read_csv(data_folder_census+'dens_2021.csv',skiprows=7,sep=',')


df_densty=df_densty[['mnemonic','2021']]


df_densty=df_densty.dropna().reset_index(drop=True)

df_densty.rename(columns={'mnemonic':'OA21CD','2021':'residents_per_square_kilometre'},inplace=True)

df_densty=df_densty[df_densty['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_densty.shape)



In [ ]:
df_hh=pd.read_csv(data_folder_census+'hh_2021.csv',skiprows=7,sep=',')


df_hh=df_hh[[x for x in df_hh.columns if x not in ['2021 output area']]]


df_hh=df_hh.dropna().reset_index(drop=True)

df_hh.rename(columns={'mnemonic':'OA21CD'},inplace=True)

df_hh=df_hh[df_hh['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_hh.shape)



In [ ]:
df_age=pd.read_csv(data_folder_census+'age_2021.csv',skiprows=4,sep=',')


df_age=df_age[[x for x in df_age.columns if x not in ['2021 output area']]]


df_age=df_age.dropna().reset_index(drop=True)

df_age.rename(columns={'mnemonic':'OA21CD'},inplace=True)

df_age=df_age[df_age['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_age.shape)



In [ ]:
df_hh_deprv=pd.read_csv(data_folder_census+'hh_deprv_2021.csv',skiprows=7,sep=',')




df_hh_deprv=df_hh_deprv[[x for x in df_hh_deprv.columns if x not in ['2021 output area']]]


df_hh_deprv=df_hh_deprv.dropna().reset_index(drop=True)

df_hh_deprv.rename(columns={'mnemonic':'OA21CD'},inplace=True)

df_hh_deprv=df_hh_deprv[df_hh_deprv['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_hh_deprv.shape)



In [ ]:
df_ethnc=pd.read_csv(data_folder_census+'ethnc_2021.csv',skiprows=7,sep=',')




df_ethnc=df_ethnc[[x for x in df_ethnc.columns if x not in ['2021 output area']]]


df_ethnc=df_ethnc.dropna().reset_index(drop=True)

df_ethnc.rename(columns={'mnemonic':'OA21CD'},inplace=True)

df_ethnc=df_ethnc[df_ethnc['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_ethnc.shape)



In [ ]:
df_health=pd.read_csv(data_folder_census+'health_2021.csv',skiprows=7,sep=',')




df_health=df_health[[x for x in df_health.columns if x not in ['2021 output area']]]


df_health=df_health.dropna().reset_index(drop=True)

df_health.rename(columns={'mnemonic':'OA21CD'},inplace=True)

df_health=df_health[df_health['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_health.shape)



In [ ]:
df_cars=pd.read_csv(data_folder_census+'cars_2021.csv',skiprows=7,sep=',')




df_cars=df_cars[[x for x in df_cars.columns if x not in ['2021 output area']]]


df_cars=df_cars.dropna().reset_index(drop=True)

df_cars.rename(columns={'mnemonic':'OA21CD'},inplace=True)

df_cars=df_cars[df_cars['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_cars.shape)



In [ ]:

df_economic_actv=pd.read_csv(data_folder_census+'economic_actv_2021.csv',skiprows=7,sep=',')




df_economic_actv=df_economic_actv[[x for x in df_economic_actv.columns if x not in ['2021 output area']]]


df_economic_actv=df_economic_actv.dropna().reset_index(drop=True)

df_economic_actv.rename(columns={'mnemonic':'OA21CD'},inplace=True)

df_economic_actv=df_economic_actv[df_economic_actv['OA21CD'].str.startswith('E')].reset_index(drop=True)


print(df_economic_actv.shape)



In [ ]:
df_census=reduce(lambda x,y: pd.merge(x,y, on='OA21CD', how='inner'),\
                 [df_densty,df_hh,df_age,df_hh_deprv,df_ethnc,df_health,df_cars,df_economic_actv]).dropna()

In [ ]:
df_census=oa_2021[['OA21CD','geometry']].merge(df_census,on=['OA21CD']).dropna()

In [ ]:
df_census=df_census.to_crs(crs_mtr)

In [ ]:
densty_ftr=['residents_per_square_kilometre']

hh_extra_ftr=['3 people in household','4 people in household', '5 people in household',\
              '6 people in household','7 people in household','8 or more people in household']


df_census['3 or more people in household']=df_census[hh_extra_ftr].sum(axis=1)


hh_ftr=['1 person in household','2 people in household','3 or more people in household']

df_census['Aged 0-24 years']=df_census[['Aged 4 years and under',\
                                                                'Aged 5 to 9 years',\
                                                                'Aged 10 to 14 years',\
                                                                'Aged 15 to 19 years',\
                                                                'Aged 20 to 24 years']].sum(axis=1)

df_census['Aged 25-49 years']=df_census[['Aged 25 to 29 years', 'Aged 30 to 34 years',\
                                                                 'Aged 35 to 39 years', 'Aged 40 to 44 years',\
                                                                 'Aged 45 to 49 years']].sum(axis=1)


df_census['Aged 50-64 years']=df_census[['Aged 50 to 54 years', 'Aged 55 to 59 years',\
                                                                 'Aged 60 to 64 years']].sum(axis=1)


df_census['Aged 65_plus years']=df_census[['Aged 65 to 69 years', 'Aged 70 to 74 years',\
                                                                   'Aged 75 to 79 years','Aged 80 to 84 years',\
                                                                   'Aged 85 years and over']].sum(axis=1)


age_ftr=['Aged 0-24 years','Aged 25-49 years','Aged 50-64 years','Aged 65_plus years']


df_census['Household is deprived in at least one dimension']=df_census[['Household is deprived in one dimension',\
                                                                        'Household is deprived in two dimensions',\
                                                                        'Household is deprived in three dimensions',\
                                                                        'Household is deprived in four dimensions']].\
sum(axis=1)


deprv_ftr=['Household is not deprived in any dimension','Household is deprived in at least one dimension']

df_census['Non_White']=df_census[['Asian, Asian British or Asian Welsh',\
                                  'Black, Black British, Black Welsh, Caribbean or African',\
                                  'Mixed or Multiple ethnic groups', 'Other ethnic group']].\
sum(axis=1)



ethnc_ftr=['White','Non_White']

df_census['Good Health']=df_census[['Very good health', 'Good health','Fair health']].sum(axis=1)


df_census['Bad Health']=df_census[['Bad health','Very bad health']].sum(axis=1)


hlth_ftr=['Good Health','Bad Health']


df_census['1 or more car or van in household']=df_census[['1 car or van in household',\
                                                                    '2 cars or vans in household',\
                                                                    '3 or more cars or vans in household']].\
sum(axis=1)




car_ftr=['No cars or vans in household','1 or more car or van in household']


df_census['Economically active']=\
df_census[['Economically active (excluding full-time students)',\
                       'Economically active and a full-time student']].sum(axis=1)




ecnmc_ftr=['Economically active', 'Economically inactive']



In [ ]:
# Normalise Features: so each feature is now a proportion

df_census_norm=df_census.copy()

df_census_norm[densty_ftr]=df_census_norm[densty_ftr]

df_census_norm[hh_ftr]=(df_census_norm[hh_ftr]/(df_census_norm[hh_ftr].sum(axis=0).sum(axis=0)))

df_census_norm[age_ftr]=(df_census_norm[age_ftr]/(df_census_norm[age_ftr].sum(axis=0).sum(axis=0)))

df_census_norm[deprv_ftr]=(df_census_norm[deprv_ftr]/(df_census_norm[deprv_ftr].sum(axis=0).sum(axis=0)))

df_census_norm[ethnc_ftr]=(df_census_norm[ethnc_ftr]/(df_census_norm[ethnc_ftr].sum(axis=0).sum(axis=0)))

df_census_norm[hlth_ftr]=(df_census_norm[hlth_ftr]/(df_census_norm[hlth_ftr].sum(axis=0).sum(axis=0)))

df_census_norm[car_ftr]=(df_census_norm[car_ftr]/(df_census_norm[car_ftr].sum(axis=0).sum(axis=0)))

df_census_norm[ecnmc_ftr]=(df_census_norm[ecnmc_ftr]/(df_census_norm[ecnmc_ftr].sum(axis=0).sum(axis=0)))


ftr_list=[hh_ftr,age_ftr,deprv_ftr,ethnc_ftr,hlth_ftr,car_ftr,ecnmc_ftr]

# Find features values for all of England
census_eng_df=[]
for ftrs in ftr_list:
    
    tmp_df=pd.DataFrame(df_census_norm[ftrs].sum()).rename(columns={0:'England'}).reset_index().\
    rename(columns={'index':'Feature'})
    
    census_eng_df.append(tmp_df)
    
    
    
# England- population density
pop_densty_eng=((df_census_norm[densty_ftr[0]]*(df_census_norm.area*(10**(-6)))).sum())/\
((df_census_norm.area*(10**(-6))).sum())

# create a list of new rows
new_df = pd.DataFrame([{'Feature': densty_ftr[0], 'England':pop_densty_eng}])



# append the new rows to the DataFrame
census_eng_df = pd.concat([pd.concat(census_eng_df,ignore_index=True),new_df], ignore_index=True)



In [ ]:
# Gather census features around the counter sites


df_census_sites_intrsctn=gpd.overlay(df_census,counters_locn, how='intersection')

df_census_sites_intrsctn[densty_ftr]=df_census_sites_intrsctn[densty_ftr]

df_census_sites_intrsctn[hh_ftr]=(df_census_sites_intrsctn[hh_ftr]/\
                                  (df_census_sites_intrsctn[hh_ftr].sum(axis=0).sum(axis=0)))

df_census_sites_intrsctn[age_ftr]=(df_census_sites_intrsctn[age_ftr]/\
                                   (df_census_sites_intrsctn[age_ftr].sum(axis=0).sum(axis=0)))

df_census_sites_intrsctn[deprv_ftr]=(df_census_sites_intrsctn[deprv_ftr]/\
                                     (df_census_sites_intrsctn[deprv_ftr].sum(axis=0).sum(axis=0)))

df_census_sites_intrsctn[ethnc_ftr]=(df_census_sites_intrsctn[ethnc_ftr]/\
                                     (df_census_sites_intrsctn[ethnc_ftr].sum(axis=0).sum(axis=0)))

df_census_sites_intrsctn[hlth_ftr]=(df_census_sites_intrsctn[hlth_ftr]/\
                                    (df_census_sites_intrsctn[hlth_ftr].sum(axis=0).sum(axis=0)))

df_census_sites_intrsctn[car_ftr]=(df_census_sites_intrsctn[car_ftr]/\
                                   (df_census_sites_intrsctn[car_ftr].sum(axis=0).sum(axis=0)))

df_census_sites_intrsctn[ecnmc_ftr]=(df_census_sites_intrsctn[ecnmc_ftr]/\
                                     (df_census_sites_intrsctn[ecnmc_ftr].sum(axis=0).sum(axis=0)))




ftr_list=[hh_ftr,age_ftr,deprv_ftr,ethnc_ftr,hlth_ftr,car_ftr,ecnmc_ftr]


census_sites_df=[]
for ftrs in ftr_list:
    
    tmp_df=pd.DataFrame(df_census_sites_intrsctn[ftrs].sum()).rename(columns={0:'Counters'}).reset_index().\
    rename(columns={'index':'Feature'})
    
    census_sites_df.append(tmp_df)
    
    
pop_densty_sites=((df_census_sites_intrsctn[densty_ftr[0]]*\
                   (df_census_sites_intrsctn.area*(10**(-6)))).sum())/\
((df_census_sites_intrsctn.area*(10**(-6))).sum())

# create a list of new rows
new_df = pd.DataFrame([{'Feature': densty_ftr[0], 'Counters':pop_densty_sites}])

# append the new rows to the DataFrame
census_sites_df = pd.concat([pd.concat(census_sites_df,ignore_index=True),new_df], ignore_index=True)





In [ ]:
# Combine the Census features for England and counter sites
census_eng_sites_df=census_eng_df.merge(census_sites_df)


census_eng_sites_df_splt=[]

for ftrs in ftr_list+[densty_ftr]:
    
    census_eng_sites_df_splt.append(census_eng_sites_df[census_eng_sites_df['Feature'].isin(ftrs)])
    
    


In [ ]:
# Comparison of the distribution of the features: England vs counter sites

f, axs = plt.subplots(nrows=round(len(census_eng_sites_df_splt)/2), ncols=2, figsize=(25,20))
# Make the axes accessible with single indexing
axs = axs.flatten()


# Start a loop over all the variables of interest
for i, col in enumerate(census_eng_sites_df_splt):
    # select the axis where the map will go
    ax = axs[i]
    # Plot the map
    col.set_index('Feature').plot(ax=ax,kind='barh',linewidth=0,fontsize=15)
    # Remove axis clutter
    #ax.set_axis_off()
    # Set the axis title to the name of variable being plotted
    #ax.set_title(col)
# Display the figure
plt.show()



In [ ]:
# Gather census features around the protected landscapes

df_census_np_intrsctn=gpd.overlay(df_census,area_of_intetest, how='intersection')

cluster_variables=densty_ftr+hh_ftr+age_ftr+deprv_ftr+ethnc_ftr+hlth_ftr+car_ftr+ecnmc_ftr

df_census_np_intrsctn=df_census_np_intrsctn[['geometry','name']+cluster_variables].copy()

# Normalise all features (now representing proportions)

df_census_np_intrsctn[densty_ftr]=df_census_np_intrsctn[densty_ftr]

df_census_np_intrsctn[hh_ftr]=(df_census_np_intrsctn[hh_ftr]/(df_census_np_intrsctn[hh_ftr].sum(axis=0).sum(axis=0)))

df_census_np_intrsctn[age_ftr]=(df_census_np_intrsctn[age_ftr]/(df_census_np_intrsctn[age_ftr].sum(axis=0).sum(axis=0)))

df_census_np_intrsctn[deprv_ftr]=(df_census_np_intrsctn[deprv_ftr]/(df_census_np_intrsctn[deprv_ftr].sum(axis=0).sum(axis=0)))

df_census_np_intrsctn[ethnc_ftr]=(df_census_np_intrsctn[ethnc_ftr]/(df_census_np_intrsctn[ethnc_ftr].sum(axis=0).sum(axis=0)))

df_census_np_intrsctn[hlth_ftr]=(df_census_np_intrsctn[hlth_ftr]/(df_census_np_intrsctn[hlth_ftr].sum(axis=0).sum(axis=0)))

df_census_np_intrsctn[car_ftr]=(df_census_np_intrsctn[car_ftr]/(df_census_np_intrsctn[car_ftr].sum(axis=0).sum(axis=0)))

df_census_np_intrsctn[ecnmc_ftr]=(df_census_np_intrsctn[ecnmc_ftr]/(df_census_np_intrsctn[ecnmc_ftr].sum(axis=0).sum(axis=0)))
                 



6. Applying clustering

In [ ]:

# one might want to scale the features before clustering: but we have not done that

df_census_np_intrsctn_scaled = df_census_np_intrsctn[[x for x in cluster_variables if x not in densty_ftr]]


In [ ]:
# Find the optimal number of clusters

Sum_of_squared_distances = []
K = range(2,20)
for num_clusters in K :
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(df_census_np_intrsctn_scaled)
    Sum_of_squared_distances.append(kmeans.inertia_)
plt.plot(K,Sum_of_squared_distances,'*-')
plt.xlabel('Values of K') 
plt.ylabel('Sum of squared distances/Inertia') 
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Initialize KMeans instance
kmeans = KMeans(n_clusters=5)
# Set the seed for reproducibility
#np.random.seed(1234)
# Run K-Means algorithm
k5cls = kmeans.fit(df_census_np_intrsctn_scaled)

# Assign labels into a column
df_census_np_intrsctn["k5cls"] = k5cls.labels_

df_census_np_intrsctn.explore(
    column="k5cls", categorical=True, legend=True
)

In [ ]:
# Group data table by cluster label and count observations
k5sizes = df_census_np_intrsctn.groupby("k5cls").size()
k5sizes

In [ ]:
df_census_np_intrsctn["area_sqm"]=df_census_np_intrsctn.area

In [ ]:
# Dissolve areas by Cluster, aggregate by summing,
# and keep column for area
areas = df_census_np_intrsctn.dissolve(by="k5cls", aggfunc="sum")["area_sqm"]
areas

In [ ]:
# Bind cluster figures in a single table
area_tracts = pd.DataFrame({"No. Tracts": k5sizes, "Area": areas})
# Convert raw values into percentages
area_tracts = area_tracts * 100 / area_tracts.sum()
# Bar plot
ax = area_tracts.plot.bar()
# Rename axes
ax.set_xlabel("Cluster labels")
ax.set_ylabel("Percentage by cluster");

area_tracts


In [ ]:
# Group table by cluster label, keep the variables used
# for clustering, and obtain their mean: distribution of Census features across all clusters
k5means = df_census_np_intrsctn.groupby("k5cls")[[x for x in cluster_variables if x not in densty_ftr]].sum()
# Transpose the table and print it rounding each value
# to three decimals
k5means.T.round(5)

7. Assigning clusters to counter sites

In [ ]:
counters_locn_copy=counters_locn.copy()

counters_locn_copy.geometry=counters_locn_copy.geometry.centroid


sites_clusters=gpd.overlay(counters_locn_copy.to_crs(df_census_np_intrsctn.crs),df_census_np_intrsctn, how='intersection')



print(sites_clusters['k5cls'].value_counts())


In [ ]:
sites_clusters_count=data_processed_sites.merge(sites_clusters[['counter','k5cls']],\
                                        how='inner',left_on=['counter'],right_on=['counter'])

sites_clusters_count[['k5cls','counter']].drop_duplicates().groupby('k5cls')['counter'].count()



#distribution of visitor numbers for each cluster
new_df = sites_clusters_count.groupby(['Date','k5cls'])['people_counter_data'].mean().unstack()
new_df.plot(style='-o',figsize=(10,2))


In [ ]:
new_df=sites_clusters_count.groupby(['Date','k5cls'])['people_counter_data'].mean().reset_index()

sns.violinplot(data=new_df, x="k5cls", y="people_counter_data")







8. Multi-group analysis for each cluster

In [ ]:

# Strava vs Visitation count regression for each cluster

new_df=sites_clusters_count.groupby(['Date','k5cls'])[['people_counter_data','total_trip_count']].mean().reset_index()


sns.lmplot(data=new_df, x="people_counter_data", y="total_trip_count", hue="k5cls")

In [ ]:

data_copy=sites_clusters_count[['Date','site','total_trip_count','people_counter_data','k5cls']]

str_df_coef=[]

for clustr in data_copy['k5cls'].unique():
    
    
    data=data_copy

    data=data[data['k5cls']==clustr].reset_index(drop=True)
    
    
    data['Month']=data['Date'].apply(lambda x: x.split('-')[1]).astype(str)
    
    dummy_month=pd.get_dummies(data['Month'],drop_first=True,prefix='Month').astype(int)
    
    del data['Month']
    
    data=pd.concat([data,dummy_month],axis=1)
    
    X=data[['total_trip_count']]#+list(dummy_month.columns)]
    X = sm.add_constant(X)
    y=data[['people_counter_data']].values
    
    print(y.shape)
    
    print(X.shape)
    
    results = sm.OLS(y,X).fit()
    
    results.summary() 
    results_as_html = results.summary().tables[1].as_html()
    
    df_coef=pd.read_html(results_as_html, header=0, index_col=0)[0]
    
    df_coef['Cluster']=clustr
    
    str_df_coef.append(df_coef)
    
    
str_df_coef=pd.concat(str_df_coef)

In [ ]:
# Finding clusters with statistically significant correlation with Strava count
sign_coef=str_df_coef[str_df_coef['P>|t|']<=0.05]

sign_coef_strava=sign_coef[sign_coef.index.isin(['total_trip_count'])]

clutrs_considered=list(sign_coef_strava['Cluster'].values)

sites_clusters=sites_clusters[sites_clusters['k5cls'].isin(clutrs_considered)]

In [ ]:
str_df_coef

In [ ]:
# Focus on sites within protected landscapes which are in the South East

sites_clusters_spefc=sites_clusters[sites_clusters['name'].isin(['Kent Downs','Surrey Hills'])]

print(sites_clusters_spefc['k5cls'].value_counts())

df_census_np_intrsctn_spefc=df_census_np_intrsctn[df_census_np_intrsctn['name'].\
                                                  isin(['Kent Downs','Surrey Hills'])]



m = df_census_np_intrsctn_spefc.to_crs(crs_deg).explore(column='k5cls',cmap='rainbow',categorical=True,legend=True)
m = sites_clusters_spefc.to_crs(crs_deg).explore(m=m,color='black')
# this is completely optional
folium.LayerControl().add_to(m)

m



In [ ]:
# split protected landscapes and counters locations into different clusters

str_all_clstr_np=[]

str_all_clstr_site=[]


for indx in range(sites_clusters_spefc['k5cls'].value_counts().shape[0]):
    
    
    
    clstr=sites_clusters_spefc['k5cls'].value_counts()[indx:indx+1].index[0]
    
    df_census_np_intrsctn_spefc_clstr=df_census_np_intrsctn_spefc[df_census_np_intrsctn_spefc['k5cls'].isin([clstr])]
    
    
    sites_spefc_clstr=sites_clusters_spefc[sites_clusters_spefc['k5cls'].isin([clstr])]
    
    str_all_clstr_np.append(df_census_np_intrsctn_spefc_clstr)
    
    str_all_clstr_site.append(sites_spefc_clstr)

In [ ]:
# 1. For each cluster, Find the polygon which contains the counter site
# 2. Find all the polygons which touch
# with the polygon found in step 1.

all_areas_touch=[]

sites_neighbrs=[]

for outr_indx in range(len(str_all_clstr_site)):
    
    sites_spefc_clstr=str_all_clstr_site[outr_indx]
    
    np_spefc_clstr=str_all_clstr_np[outr_indx]
    
    for indx in range(len(sites_spefc_clstr)):
        
        
        
        row=sites_spefc_clstr.iloc[indx:indx+1,:]
    
   
    
        geom=np_spefc_clstr


    

        site_ctnd=gpd.GeoDataFrame(gpd.sjoin(geom, row, how='inner', predicate='contains').geometry)
    
   
    
  
    
    
        all_areas_touch.append(pd.concat([gpd.sjoin(np_spefc_clstr, site_ctnd, how='inner',\
                                                    predicate='touches'),geom[geom.index==site_ctnd.index[0]]]))
    
        sites_neighbrs.append(row)

In [ ]:
#sites within protected landscapes
sites_within_np=pd.concat(sites_neighbrs)

#protected landscapes touching counter sites
areas_touch_sites=pd.concat(all_areas_touch)

In [ ]:
sites_within_np.to_crs(crs_deg).geometry

In [ ]:
# visualisation
# for each cluster, polygon containing the counter site and all polygons touching it.

m = areas_touch_sites.to_crs(crs_deg).explore(column='k5cls',cmap='rainbow',categorical=True,legend=True)
m = sites_within_np.to_crs(crs_deg).explore(m=m,color='black')
# this is completely optional
folium.LayerControl().add_to(m)

m

9. Making estimates for protected spaces

In [ ]:
df_strava_np_shp=[]#gpd.read_file('./data/strava_data_protected_sites')

shp_files=['./data/strava_data_protected_sites/'+x for x in os.listdir('./data/strava_data_protected_sites')\
 if x.split('.')[1]=='shp']


df_strava_np_shp=[gpd.read_file(x) for x in shp_files]

df_strava_np_shp=pd.concat(df_strava_np_shp).reset_index(drop=True)




df_strava_np_shp=df_strava_np_shp.to_crs(crs_mtr)

df_strava_np_shp=df_strava_np_shp.drop_duplicates(subset=['edgeUID']).reset_index(drop=True)

# Intersection between Strava trails and polygons within protected landscape




df_strava_np_shp_intrsctn=gpd.overlay(df_strava_np_shp, areas_touch_sites[['geometry','name','k5cls']],\
                                      how='intersection')

df_strava_np_shp_intrsctn=df_strava_np_shp_intrsctn[['edgeUID','osmId','geometry','name','k5cls']]



m = areas_touch_sites.to_crs(crs_deg).explore(column='k5cls',cmap='rainbow',categorical=True,legend=True)
m=df_strava_np_shp_intrsctn.to_crs(crs_deg).explore(m=m,color='yellow')

m


In [ ]:


csv_files=['./data/strava_data_protected_sites/'+x for x in os.listdir('./data/strava_data_protected_sites')\
 if x.split('.')[1]=='csv']


df_strava_np_csv=[pd.read_csv(x) for x in csv_files]

df_strava_np_csv=pd.concat(df_strava_np_csv).reset_index(drop=True)

df_strava_np_csv=df_strava_np_csv.drop_duplicates(subset=['edge_uid','month']).reset_index(drop=True)

df_strava_np_csv

In [ ]:
strava_edge_count_np=df_strava_np_csv.merge(df_strava_np_shp_intrsctn,left_on=['edge_uid'],\
                                            right_on=['edgeUID'],how='inner').\
groupby(['month','edge_uid'])['total_trip_count'].mean().reset_index()


strava_edge_count_np=df_strava_np_shp_intrsctn.merge(strava_edge_count_np,\
                                                     left_on=['edgeUID'],right_on=['edge_uid'],how='inner')

strava_edge_count_np=strava_edge_count_np.merge(sign_coef_strava.reset_index(drop=True),\
                                                left_on=['k5cls'],right_on=['Cluster'])

strava_edge_count_np['estimated_people_count']=strava_edge_count_np['total_trip_count']*\
strava_edge_count_np['coef']



In [ ]:
sign_coef_strava

In [ ]:
estm_edge_count_np_mean=strava_edge_count_np.groupby(['edgeUID','name'])['estimated_people_count'].mean().\
reset_index()


estm_edge_count_np_mean=strava_edge_count_np[['edgeUID','geometry']].drop_duplicates().\
merge(estm_edge_count_np_mean,left_on=['edgeUID'],right_on=['edgeUID'],how='inner')


In [ ]:
# Average over time 

m = areas_touch_sites.to_crs(crs_deg).explore(column='name',categorical=True)
m=estm_edge_count_np_mean.explore(m=m,column='estimated_people_count',cmap='rainbow')
m



In [ ]:
# Average across all edges

estimated_count_np=strava_edge_count_np.groupby(['month','name'])['estimated_people_count'].mean().reset_index()

px.bar(estimated_count_np,x='month',y='estimated_people_count',color='name',barmode='stack')
